In [0]:
# Databricks-ready PySpark code for dataset: ott_user_behavior_1
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.recommendation import ALS
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

spark = SparkSession.builder.appName("OTT_Analytics").getOrCreate()

# ---------- 1) Load dataset (try Spark table, else CSV fallback) ----------
dataset_name = "ott_user_behavior_1"

try:
    df = spark.table(dataset_name)
    print(f"Loaded Spark table: {dataset_name}")
except Exception as e:
    print(f"Table {dataset_name} not found, trying CSV at /FileStore/tables/{dataset_name}.csv")
    df = spark.read.option("header", "true").option("inferSchema", "true") \
            .csv(f"/FileStore/tables/{dataset_name}.csv")

# quick clean / cast commonly-needed columns
# adjust column names if your schema differs (case-sensitive in Spark)
# Ensure numeric columns are numeric
num_cols = ["age", "avg_watch_time_per_day", "total_watch_time", "watch_duration",
            "completion_rate", "rating_given", "total_sessions_per_week",
            "avg_stream_quality", "network_speed_mbps", "buffering_count",
            "monthly_spend", "peak_watch_hour", "churn_flag"]
for c in num_cols:
    if c in df.columns:
        df = df.withColumn(c, col(c).cast(DoubleType()))

# Fill NA for numeric cols with 0 for safe model training (simple strategy)
numeric_present = [c for c in num_cols if c in df.columns]
df = df.fillna({c: 0.0 for c in numeric_present})

Loaded Spark table: ott_user_behavior_1


In [0]:
# ---------- Use Case 1: User Viewing Pattern Analysis ----------
# Avg daily watch time and total watch time by region and subscription_type
viewing_pattern = (df.groupBy("region", "subscription_type")
                     .agg(
                         round(avg("avg_watch_time_per_day"),2).alias("avg_watch_time_per_day"),
                         round(avg("total_watch_time"),2).alias("avg_total_watch_time"),
                         countDistinct("user_id").alias("unique_users")
                     ).orderBy(desc("avg_watch_time_per_day"))
                  )

display(viewing_pattern)  # Databricks display

# Top genres by total watch time
if "genre" in df.columns:
    genre_trends = (df.groupBy("genre")
                     .agg(round(sum("total_watch_time"),2).alias("total_watch_time"))
                     .orderBy(desc("total_watch_time")))
    display(genre_trends.limit(10))

# Device type preference
if "device_type" in df.columns:
    device_usage = df.groupBy("device_type").count().orderBy(desc("count"))
    display(device_usage)

region,subscription_type,avg_watch_time_per_day,avg_total_watch_time,unique_users
Karnataka,Standard,112.15,6114.73,222
Maharashtra,Basic,111.3,6022.0,191
Bihar,Standard,109.92,5948.74,206
Tamil Nadu,Premium,109.73,5729.66,212
Delhi,Standard,108.72,6027.83,208
Kerala,Standard,107.77,6053.64,206
Delhi,Premium,107.65,5661.59,209
Uttar Pradesh,Premium,106.86,5892.33,179
Maharashtra,Premium,106.79,5704.6,202
Bihar,Basic,105.78,5812.22,210


genre,total_watch_time
Documentary,3790601.29
Romance,3740899.35
Horror,3625878.02
Drama,3605819.13
Comedy,3584950.13
Sci-Fi,3528333.74
Thriller,3510925.05
Action,3291081.25


Databricks visualization. Run in Databricks to view.

device_type,count
Laptop,1289
Tablet,1279
Mobile,1233
Smart TV,1199


Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.sql.functions import (
    col, avg, sum as _sum, countDistinct, round, hour, dayofweek, desc
)

# ✅ Load dataset from Databricks Catalog
df = spark.table("workspace.default.ott_user_behavior_1")

# ---------- REGION & SUBSCRIPTION WATCH TIME ----------
viewing_pattern = (
    df.groupBy("region", "subscription_type")
      .agg(
          round(avg("avg_watch_time_per_day"), 2).alias("avg_watch_time_per_day"),
          round(avg("total_watch_time"), 2).alias("avg_total_watch_time"),
          countDistinct("user_id").alias("unique_users")
      )
      .orderBy(desc("avg_watch_time_per_day"))
)

display(viewing_pattern)

# ---------- TOP GENRES BY TOTAL WATCH TIME ----------
if "genre" in df.columns:
    genre_trends = (
        df.groupBy("genre")
          .agg(round(_sum("total_watch_time"), 2).alias("total_watch_time"))
          .orderBy(desc("total_watch_time"))
    )
    display(genre_trends.limit(10))

# ---------- DEVICE TYPE PREFERENCE ----------
if "device_type" in df.columns:
    device_usage = (
        df.groupBy("device_type")
          .count()
          .orderBy(desc("count"))
    )
    display(device_usage)

# ---------- USER ENGAGEMENT SUMMARY ----------
if all(x in df.columns for x in ["user_id", "total_watch_time", "completion_rate"]):
    user_engagement = (
        df.groupBy("user_id")
          .agg(
              round(avg("completion_rate"), 2).alias("avg_completion_rate"),
              round(_sum("total_watch_time"), 2).alias("total_watch_time"),
              countDistinct("content_id").alias("unique_contents_watched")
          )
          .orderBy(desc("total_watch_time"))
    )
    display(user_engagement)

# ---------- HOURLY WATCHING PATTERN ----------
if "watch_date" in df.columns:
    df = df.withColumn("watch_hour", hour(col("watch_date")))
    hourly_pattern = (
        df.groupBy("watch_hour")
          .agg(countDistinct("user_id").alias("active_users"))
          .orderBy("watch_hour")
    )
    display(hourly_pattern)

# ---------- WEEKLY VIEWING TREND ----------
if "watch_date" in df.columns:
    df = df.withColumn("day_of_week", dayofweek(col("watch_date")))
    weekly_pattern = (
        df.groupBy("day_of_week")
          .agg(
              countDistinct("user_id").alias("unique_users"),
              round(avg("total_watch_time"), 2).alias("avg_watch_time")
          )
          .orderBy("day_of_week")
    )
    display(weekly_pattern)

# ---------- OPTIONAL: SAVE AS DASHBOARD TABLES ----------
viewing_pattern.write.mode("overwrite").format("delta").saveAsTable("ott_viewing_pattern_summary")
if "genre" in df.columns:
    genre_trends.write.mode("overwrite").format("delta").saveAsTable("ott_genre_trend_summary")
if "device_type" in df.columns:
    device_usage.write.mode("overwrite").format("delta").saveAsTable("ott_device_usage_summary")

print("✅ Use Case 1 Analysis Complete — summaries stored for dashboard visualization.")

region,subscription_type,avg_watch_time_per_day,avg_total_watch_time,unique_users
Karnataka,Standard,112.15,6114.73,222
Maharashtra,Basic,111.3,6022.0,191
Bihar,Standard,109.92,5948.74,206
Tamil Nadu,Premium,109.73,5729.66,212
Delhi,Standard,108.72,6027.83,208
Kerala,Standard,107.77,6053.64,206
Delhi,Premium,107.65,5661.59,209
Uttar Pradesh,Premium,106.86,5892.33,179
Maharashtra,Premium,106.79,5704.6,202
Bihar,Basic,105.78,5812.22,210


Databricks visualization. Run in Databricks to view.

genre,total_watch_time
Documentary,3790601.29
Romance,3740899.35
Horror,3625878.02
Drama,3605819.13
Comedy,3584950.13
Sci-Fi,3528333.74
Thriller,3510925.05
Action,3291081.25


Databricks visualization. Run in Databricks to view.

device_type,count
Laptop,1289
Tablet,1279
Mobile,1233
Smart TV,1199


...

Databricks visualization. Run in Databricks to view.

watch_hour,active_users
0,5000


day_of_week,unique_users,avg_watch_time
1,726,5835.31
2,707,5696.09
3,741,5886.78
4,696,5496.49
5,696,5656.35
6,674,5700.59
7,760,5852.95


Databricks visualization. Run in Databricks to view.

✅ Use Case 1 Analysis Complete — summaries stored for dashboard visualization.
